# Week 1

### Background Knowledge

1. Out of million nucleotides, there is a sequence of 500-1000 long nucleotides called **OriC** where the replication begins.
2. DNA replication requires protein called **DnaA** that binds to (usually) 9-nucleotide long segment known as Dna A box. There are multiple instances of Dna A box within OriC.
3. First we have to locate OriC (**700/1,000,000s**) and find correct Dna A boxes (**5/700**)

### Defining problems


**1. Finding origin of gene replication**
<br>
&emsp; *Input: A genome*
<br>
&emsp; *Output: the location of origin (**oriC** in genome)*
  
-> <font color=red>This is the ill defined CS problem</font>

**2. Finding a hidden message in a string** <br>
&emsp; *Input: Text* <br>
&emsp; *Output: Hidden message*

-> <font color=red>This is still not clear</font>

**3. Finding most frequent k-mers in a string** <br>
&emsp; *Input: A string genome, integer k* <br>
&emsp; *Output: All of the most frequent k-mers in text*

-> <font color=blue>This is now clear!</font>

**4. Clump finding problem** <br>
&emsp; *Input: A string genome, <br>
&emsp;&emsp;&emsp;&ensp; integer k (length of pattern),<br>
&emsp;&emsp;&emsp;&ensp; integer l (window length),<br>
&emsp;&emsp;&emsp;&ensp; integer t (# of patterns in the genome)*

&emsp; *Output: All k-mers forming **l-t clumps** in the genome*


In [1]:
with open('data/week_1/vibrio_cholerae.txt', 'r') as f:
    genome = f.read()

In [2]:
print("The length is genome is %d nucleotides" % len(genome))
print("Number of occurences of A: %d" % genome.count("A"))
print("Number of occurences of C: %d" % genome.count("C"))
print("Number of occurences of T: %d" % genome.count("T"))
print("Number of occurences of G: %d" % genome.count("G"))

The length is genome is 1108250 nucleotides
Number of occurences of A: 293942
Number of occurences of C: 263573
Number of occurences of T: 294711
Number of occurences of G: 256024


In [11]:
def Count(string, substring):
    count = 0
    for i in range(len(string) - len(substring) + 1):
        if string[i:i+len(substring)] == substring:
            count += 1
    return count

Count('CGCGATACGTTACATACATGATAGACCGCGCGCGATCATATCGCGATTATC', 'CGCG')

with open('data/week_1/dataset_2_7.txt', 'r') as f:
    sample_problem = f.read()
    
string, substring, _ = sample_problem.split('\n')
Count(string, substring)

27

In [12]:
def FrequentWords(text, k):
    word_counter = {}
    
    for i in range(len(text) - k + 1):
        word = text[i:i+k]
        if word in word_counter:
            word_counter[word] += 1
        else:
            word_counter[word] = 1
    
    max_count = max(list(word_counter.values()))
    
    return [word for word, count in word_counter.items() 
            if count == max_count], max_count

FrequentWords('CGCCTAAATAGCCTCGCGGAGCCTTATGTCATACTCGTCCT', 3)

with open('data/week_1/dataset_2_10.txt', 'r') as f:
    sample_problem = f.read()
    
text, k, _ = sample_problem.split('\n')
k = int(k)

FrequentWords(text, k)

(['GCGTGGTTGGG'], 14)

In [13]:
FrequentWords(genome, 9)

(['GCGTTTGTT'], 128)

In [21]:
def ReverseComplement(sequence):
    dic = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C'}
    
    return ''.join([dic[sequence[len(sequence)-i-1]] for i in range(len(sequence))])

In [74]:
ReverseComplement('AAAACCCGGT')

with open('data/week_1/dataset_3_2.txt', 'r') as f:
    sample_problem = f.read()
    
sequence = sample_problem.strip()
#print(ReverseComplement(sequence))

print(ReverseComplement('GCTAGCT'))

AGCTAGC


In [45]:
def PatternMatching(substring, string):
    match_index = []
    for i in range(len(string) - len(substring) + 1):
        if string[i:i+len(substring)] == substring:
            match_index.append(i)
            
    return match_index

In [75]:
PatternMatching('ATAT', 'GATATATGCATATACTT')

with open('data/week_1/dataset_3_5.txt', 'r') as f:
    sample_problem = f.read()
    
substring, string = sample_problem.split()

#print(' '.join([str(i) for i in PatternMatching(substring, string)]))
print(PatternMatching('ATA', 'GACGATATACGACGATA'))

[4, 6, 14]


In [67]:
def FindClump(genome, k, L, t):
    valid_clumps = []
    for w in range(len(genome) - L + 1):
        window = genome[w:w+L]
        clump_counter = {}

        for i in range(L - k + 1):
            clump = window[i:i+k]
            if clump in clump_counter:
                clump_counter[clump] += 1
            else:
                clump_counter[clump] = 1
    
        valid_clumps.extend([clump for clump, count 
                             in clump_counter.items()
                             if count >= t])
    return list(set(valid_clumps))

In [73]:
FindClump('CGGACTCGACAGATGTGAAGAACGACAATGTGAAGACTCGACACGACAGAGTGAAGAGAAGAGGAAACATTGTAA', 
          5, 50, 4)

with open('data/week_1/dataset_4_5.txt', 'r') as f:
    sample_problem = f.read()
    
genome, k, L, t = sample_problem.split()
k = int(k)
L = int(L)
t = int(t)

print(' '.join(FindClump(genome, k, L, t)))

AAGTGTAA CTCGTTAC


**Frequency Array - Make FrequentWords faster**


In [120]:
def PatternToNumber(genome):
    number = 0
    
    for i in range(len(genome)):
        nuc = genome[i]
        nuc_index = 'ACGT'.index(nuc)
        
        number += nuc_index * (4 ** (len(genome)-i-1))
        
    return number

def NumberToPattern(number, genome_length):
    genome = ''
    for i in range(genome_length):
        base = 4 ** (genome_length - i - 1)
        genome += 'ACGT'[int(number // base)]
        number = number % base
        
    return genome


def FrequencyArray(genome, k):
    frequencyArray = [0] * (4 ** k)
    
    for i in range(len(genome) - k + 1):
        frequencyArray[PatternToNumber(genome[i:i+k])] += 1
        
    return frequencyArray
        

In [121]:
PatternToNumber('ATGCAA')
NumberToPattern(5437, 8)
FrequencyArray('ACGCGGCTCTGAAA', 2)

[2, 1, 0, 0, 0, 0, 2, 2, 1, 2, 1, 0, 0, 1, 1, 0]

In [126]:
with open('data/week_1/dataset_2994_5.txt', 'r') as f:
    sample_problem = f.read()
    
genome, k = sample_problem.split()
k = int(k)

In [128]:
print(' '.join(str(i) for i in FrequencyArray(genome, k)))

0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [129]:
print(k)

8
